# Yahoo Finance e API do google sheets

# Objetivo
A ideia do projeto é usar a API do sheets para que com os dados gerados pelo Yahoo Finance para atualizar a planilha com os seguintes dados:

- *Date*: Data;
- *Open*: O valor da ação na abertura (em dólar se for nos EUA e em real se for BR);
- *High*: O maior valor da ação naquela data;
- *Low*: O menor valor da ação naquela data;
- *Close*: O valor da ação no fechamento;
- *Volume*: Volume de ações da empresa.

# Definindo as funções para a obtenção dos dados das ações

Aqui veremos as funções que usaremos *get_stocks* para obter as informação da ação e a função *stocks_df* para armazenar esses dados num dataframe para fazer o update na planilha no Google Sheets

In [1]:
import yfinance as yf
import pandas as pd

In [2]:
# Função para obter os dados de cada ação
def get_stocks(ticker: str, country: str):
    stock_data = yf.Ticker(ticker).history(period='1day')
    stock_data['Stock'] = [f'{ticker}']
    stock_data['Country'] = [f'{country}']
    stock_data = stock_data.reset_index().drop(columns=['Dividends', 'Stock Splits'])
    stock_data['Date'] = stock_data['Date'].dt.strftime('%d-%m-%Y')
    return stock_data

# Função para fazer a ligação entre a lista de ações e o dataframe que será criado
def stocks_df(list: dict):
    dfs = []
    for chave, valor in list.items():
        for x in valor:  
            df = get_stocks(x, chave)
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


In [3]:
# Agora um dicionário com as ações com maiores volumes no Brasil e nos EUA

stocks = {
    'BR': ['PETR4.SA', 'VALE3.SA', 'BBDC4.SA', 'ITUB4.SA', 'B3SA3.SA', 'MGLU3.SA', 'ABEV3.SA', 'WEGE3.SA', 'BBAS3.SA', 'RENT3.SA'],
    'US': ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TSLA', 'META', 'GOOG', 'NVDA']
}

In [5]:
# API Google
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "insira aqui a id da planilha do sheets"
SAMPLE_RANGE_NAME = "Data!A:G"


def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
      sheet.values()
      .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
      .execute()
    )
    values = result.get("values", [])
    #print(values)
    valor = stocks_df(stocks).values.tolist()
    

    # Update values
    result = (
      sheet.values()
      .update(spreadsheetId= SAMPLE_SPREADSHEET_ID, range=f'Data!A{len(values)+1}', valueInputOption="RAW", body={"values": valor})
      .execute()
    )

  except HttpError as err:
    print(err)



if __name__ == "__main__":
  main() 

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=886193867961-irpfb7kqmrtvdrart87t1g0uo6ivsv8r.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62438%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=YHAFsNwENufKS5yvPw16b3PWgs3VgN&access_type=offline
[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Stock'], ['12-01-2024', '186,0599976', '186,7400055', '185,1900024', '185,8800049', '19504032', 'AAPL'], ['12-01-2024', '144,3399963', '144,7398987', '143,3600006', '144,5650024', '7086644', 'GOOG'], ['12-01-2024', '385,4899902', '388,480011', '384,6499939', '388,3299866', '11111116', 'MSFT'], ['15-01-2024', '37,99000168', '38,16999817', '37,86000061', '38,13999939', '1628300', 'PETR4.SA'], ['15-01-2024', '71,22000122', '71,30000305', '70,69999695', '70,79000092', '2054300', 'VALE3.SA'], ['15-01-2024', '15,75', '15,77999973', '15,64999962', '15,69999981', '938400', 'BBDC4.SA'], 